In [20]:
#### label 정보
#### (배경 - 0 / 몸통 - 1 / 오른손 - 2 / 왼손 - 3 / 왼발 - 4 / 오른발 - 5 / 오른쪽 허벅지 - 6 / 왼쪽 허벅지 - 7 / 오른쪽 종아리 - 8 / 왼쪽 종아리 - 9 / 왼쪽 팔 - 10 / 
#### 오른쪽 팔 - 11 / 왼쪽 전완 - 12 / 오른쪽 전완 - 13 / 머리 - 14)
#### 해당 값이 RGB 값임 ex) 배경 0,0,0
### mask 확장자 png input image 확장자 jpg 해깔리지 말것

In [21]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)
import mmcv
import os.path as osp
import numpy as np
import skimage.io as iio

1.9.0 True
0.18.0


In [22]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [23]:
classes = ('bg', 'body', 'right_hand', 'left_hand', 'left_leg', 'right_reg', 'right_thigh', 'left_thigh','right_calf','left_calf'
           ,'left_arm','right_arm','left_forearm','right_forearm','head')

In [24]:
palette = [[0,0,0], [1,1,1], [2,2,2], [3,3,3],[4,4,4], [5,5,5], [6,6,6], [7,7,7], [8,8,8], [9,9,9], [10,10,10], [11,11,11], [12,12,12], [13,13,13], [14,14,14]]

In [25]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class body_seg(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None


KeyError: 'body_seg is already registered in dataset'

In [26]:
from mmcv import Config

In [27]:
from mmseg.apis import set_random_seed
cfg = Config.fromfile('mmsegmentation/configs/swin/upernet_swin_small_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.py')

In [28]:
cfg.dataset_type = 'body_seg'
cfg.data_root = '/root'


# Since we use ony one GPU, BN is used instead of SyncBN
## encoding 부분은 layer normalization 
## encoding 이후 ecoder part 부터는 batchnormalization
cfg.norm_cfg = dict(type='LN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.auxiliary_head.norm_cfg = dict(type='BN', requires_grad=True)


# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 15
cfg.model.auxiliary_head.num_classes = 15

cfg.data.samples_per_gpu = 24
cfg.data.workers_per_gpu = 2

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (224, 224)

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(224, 224),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]



checkpoint_file = 'pretrained/upernet_swin_small_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.pth'
cfg.load_from = checkpoint_file

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/train_small'

# 1epoch 대략 43000
cfg.runner.max_iters = 10833*50
cfg.log_config.interval = 3610
cfg.evaluation.interval = 10833
cfg.checkpoint_config.interval = 10833

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='LN', requires_grad=True)
backbone_norm_cfg = dict(type='LN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='pretrain/swin_small_patch4_window7_224.pth',
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=224,
        embed_dims=96,
        patch_size=4,
        window_size=7,
        mlp_ratio=4,
        depths=[2, 2, 18, 2],
        num_heads=[3, 6, 12, 24],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=dict(type='LN', requires_grad=True)),
    decode_head=dict(
        type='UPerHead',
        in_channels=[96, 192, 384, 768],
        in_index=[0, 1, 2, 3],
        pool_scales=(1, 2, 3, 6),
        channels=512,
        dropout_ratio=0.1,
        num_c

In [29]:

from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor



# Build the detector

# config_file = 'mmsegmentation/configs/swin/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K.py'
model = init_segmentor(cfg, checkpoint_file, device='cuda:0')
# model = build_segmentor(
#     cfg.model, test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = classes

Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([150, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([15, 512, 1, 1]).
size mismatch for decode_head.conv_seg.bias: copying a param with shape torch.Size([150]) from checkpoint, the shape in current model is torch.Size([15]).
size mismatch for auxiliary_head.conv_seg.weight: copying a param with shape torch.Size([150, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([15, 256, 1, 1]).
size mismatch for auxiliary_head.conv_seg.bias: copying a param with shape torch.Size([150]) from checkpoint, the shape in current model is torch.Size([15]).


In [30]:
# checkpoint에 config 정보까지 같이 들어 있으나 PALLET 정보가 meta에서 빠져있어서
# weight만 따로 불러서 weight만 load 해줌..
datas=torch.load('work_dirs/train_small/iter_151662.pth')['state_dict']

In [31]:
model.load_state_dict(datas)
model.eval()

EncoderDecoder(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (adap_padding): AdaptivePadding()
      (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (drop_after_pos): Dropout(p=0.0, inplace=False)
    (stages): ModuleList(
      (0): SwinBlockSequence(
        (blocks): ModuleList(
          (0): SwinBlock(
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): ShiftWindowMSA(
              (w_msa): WindowMSA(
                (qkv): Linear(in_features=96, out_features=288, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=96, out_features=96, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop): DropPath()
            )
            (norm2): LayerNorm((96,), eps=1e-05, elementwi

In [32]:
# validation set 받아오기 (나중에는 .py 파일로해서 경로 받을수 있게 해야함 제출양식..)
val_ls=[]
file = open("splits/val.txt", "r")
while True:
    line = file.readline()
    if not line:
        break
    val_ls.append(line.strip())

file.close()

In [33]:
model.cfg = cfg

In [34]:
result = inference_segmentor(model, 'val2014/' + val_ls[0] + '.jpg')

In [35]:
pallete_np=np.array(palette)

In [36]:
img = pallete_np[result[0]]

In [37]:
type(img.dtype)



numpy.dtype[int64]

In [38]:
iio.imsave('tttt.jpg',img)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: tttt.jpg is a low contrast image
  """Entry point for launching an IPython kernel.
Lossy conversion from int64 to uint8. Range [0, 14]. Convert image to uint8 prior to saving to suppress this warning.
